In [ ]:
# 1.Preparing corpus - 의사록 다운로드

import requests
import bs4

def download_pdf(url,file_name):
    with open(file_name,"wb") as file:
        response = requests.get(url)
        file.write(response.content)

pdf_urls = []
pdf_dates = []

# 1쪽부터 29쪽까지 다운로드
for i in range(1,30):
    url = "https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}".format(i)
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, "html.parser")
    pdf_results = soup.select("div.fileGoupBox")
    date_results = soup.select("span.titlesub")
    for result in pdf_results:
        if result.find("a")["title"][-3:]=='pdf':
            pdf_urls.append(result.find("a")["href"])
        else:
            pdf_urls.append(result.find_all("a")[2]["href"])
    for result in date_results:
        pdf_dates.append(result.text[24:35])

base_url = "https://www.bok.or.kr"
for i in range(len(pdf_urls)):
    pdf_urls[i]=base_url + pdf_urls[i]

for i in range(len(pdf_urls)):
    download_pdf(pdf_urls[i],"/content/drive/MyDrive/minutes/{}.pdf".format(pdf_dates[i]))

In [ ]:
# 의사록 전처리 후 파싱(.pdf -> .csv)

# 문장 전처리 함수
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

import re
# 의사록 섹션별 전처리 함수
def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [ ]:
pip install fitz

In [ ]:
# 각 의사록 섹션 내용 파싱(.pdf -> .csv)

import fitz
contents=[]
dates=[]

import os
os.chdir(r'D:\JO\asiaeconomy\kdigital\hong\project\minutes')
from glob import glob
pdfs = glob('*.pdf')
for pdf in pdfs:
    doc = fitz.open('/content/drive/MyDrive/minutes/'+pdf) #for문 하나씩 추가
    page_count = doc.pageCount
    page=0
    content_str=""
    while(page<page_count):
        p = doc.loadPage(page)
        content_str = content_str+p.getText()
        page +=1
    sections, section_texts = preprocess_minutes(content_str)
    dates.append(pdf[0:8])
    contents.append(section_texts[1]+section_texts[2])  #의사록 섹션 2, 섹션3을 하나로 합쳐서 append

In [ ]:
# 데이터 프레임에 date, content 저장

import pandas as pd
for i in range(len(dates)):
    dates[i] = dates[i][0:4]+'.'+dates[i][4:6]+'.'+dates[i][6:8]
for i in range(len(contents)):
    contents[i] = "@@@".join(contents[i])
minutes_data = {
    'date':dates,
    'content':contents
}
minutes_data = pd.DataFrame(minutes_data)
for i in range(len(minutes_data)):
    if minutes_data.loc[i]['content'] == '':
        minutes_data.drop(i,inplace=True)

In [ ]:
from ekonlpy.sentiment import MPCK
mpck = MPCK()

ModuleNotFoundError: ignored

In [ ]:
# 의사록의 content를 n-gram화한 후 .csv파일로 저장

def make_ngram(df):
    try:
        tokens=mpck.tokenize(df['content'])
        ngrams=mpck.ngramize(tokens)
        tokens_text=",".join(tokens)
        ngrams_text=",".join(ngrams)
        total_text=tokens_text+","+ngrams_text
    except Exception as e:
        print(e)
        total_text=""
    return total_text

minutes_data['ngrams']=minutes_data.apply(make_ngram,axis=1)
minutes_data.to_csv("minutes_data_save.csv",encoding='utf-8')